In [ ]:
import os
import pandas as pd
import numpy as np
import logging
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
logging.basicConfig(level=logging.INFO)
logging = logging.getLogger(__name__)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_dir = 'drive/My Drive/knowledge engineering/assignments/assignment_4/'

Mounted at /content/drive


In [ ]:
class ProcessTable:
    def __init__(self, table, table_name):
        self.table = table
        self.table_name = table_name

    def create_sentence(self, row):        
        sentence = ' '.join(row[:-1])
        sentence = sentence.lower()
        return sentence
    
    def preprocess_row(self, row):
        row = row.lower()
        row = row.split('\t')
        row = [i for i in row if len(i) != 0]
        id = row[-1].split('\n')[0]
        return row, id

    def filter_words(self, sentence):
        stop_words = set(stopwords.words('english'))
        # filtered_words = [i for i in row[:-1] if i not in stop_words]
        filtered_words = [i for i in sentence.split(' ') if i not in stop_words]
        return filtered_words

    def iterate_table(self):
        new_table = []
        for row in self.table[1:]:
            row, id = self.preprocess_row(row)
            sentence = self.create_sentence(row)
            filtered_words = self.filter_words(sentence)
            new_table.append([id, sentence, filtered_words, self.table_name, 'fact'])
        return new_table

    def main(self):
        table = self.iterate_table()
        return table

In [ ]:
class EntryPoint:
    def __init__(self, input_directory, output_directory):
        self.input_directory = input_directory
        self.output_directory = output_directory
    
    def get_tables(self):
        file_list = os.listdir(self.input_directory)
        return file_list

    def load_table(self, file_path):        
        with open(file_path) as f:
            table = f.readlines()   
        return table     

    def iterate_tables(self):
        sentence_node_table = [['fact_id:ID', 'fact', 'filtered_fact', 'table_name', 
                                ':LABEL']]
        for file_name in self.get_tables():
            if file_name.endswith('tsv'):
                file_path = os.path.join(self.input_directory, file_name)
                temp_sentence_node_table = self.load_table(file_path)
                sentence_node_table.extend(ProcessTable(temp_sentence_node_table, 
                                                file_name.split('.')[0]).main())
        return sentence_node_table

    def to_dataframe(self, data):
        data = pd.DataFrame(data)
        data.columns = data.iloc[0]
        data.drop(0, axis=0, inplace=True)
        for column in data.columns:
            if ':ID' in column:
                data.drop_duplicates(column, inplace=True)
        return data

    def write_to_csv(self, file_name, data):
        logging.info(f'Writing {file_name} to file')
        file_path = os.path.join(self.output_directory, file_name)
        data.to_csv(file_path, index=False)

    def main(self):
        sentence_node_table = self.iterate_tables()
        shared_term_relationships = Relationships(sentence_node_table).main()
        sentence_node_table = self.to_dataframe(sentence_node_table)
        shared_term_relationships = self.to_dataframe(shared_term_relationships)
        self.write_to_csv('sentence_node_table.csv', sentence_node_table)
        self.write_to_csv('shared_term_relationships.csv', shared_term_relationships)
        return sentence_node_table, shared_term_relationships

In [ ]:
class Relationships:
    def __init__(self, table):
        self.table = table

    def iterate_table(self):
        shared_terms_relationship = [[':START_ID', ':END_ID', ':TYPE', 'shared_term']]
        for row in self.table[1:]:
            for term in row[2]:
                for row_2 in self.table[1:]:
                    if term in row_2[2]:
                        shared_terms_relationship.append([row[0], row_2[0], 'SHARED_TERM', term])
        return shared_terms_relationship

    def main(self):
        return self.iterate_table()

In [ ]:
input_directory = os.path.join(base_dir, 'data/worldtree_full/tsv/tables')
output_directory = os.path.join(base_dir, 'data/delta_files')
sentence_node_table, shared_term_relationships = EntryPoint(
                                    input_directory, 
                                    output_directory
                                    ).main()

INFO:__main__:Writing sentence_node_table.csv to file
INFO:__main__:Writing shared_term_relationships.csv to file
